In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [40]:
from scipy.misc import toimage

SIZE = 1024

# Chaos game coloring of the Gasket

## Normal Mappings

In [6]:
transform = array([[0,0],[1,0], [1/2,sqrt(3)/2]]).T;

In [12]:
img = np.zeros((SIZE+1, SIZE+1), dtype=float32)

c = array([0,0]) #initial point

for i in range(100000):
    s = random.randint(3)
    c = (c+transform[:,s])/2
    img[SIZE-1-int(c[1]*SIZE),int(c[0]*SIZE)] = 1

toimage(img).show()

## Using color mappings (for transformation) and alpha (for counts)

In [111]:
img = np.zeros((4, SIZE+1, SIZE+1), dtype=float32)
img[3,:,:].fill(1)

colors = eye(3); #colors on RGB channels

curr_col = [1,0,0] #we start on bottom-left which is R-channel

c = array([0,0]) #initial point

for i in range(100000):
    s = random.randint(3)
    c = (c+transform[:,s])/2
    
    row = SIZE-1-int(c[1]*SIZE)
    col = int(c[0]*SIZE)
    
    curr_col = (curr_col+colors[s,:])/2
    
    img[0:3,row,col] = curr_col
    img[3,row,col] += 1

for i in range(0,3):
    img[i,:,:] *= log(img[3,:,:])/img[3,:,:]
        
toimage(img[0:3,:,:]).show()

## Let's get weird

In [141]:
def sphere_transform(p):
    return p/(p @ p+0.001)

def sin_transform(p):
    return sin(p)

def swirl_transform(p):
    r2 = p @ p
    x = p[0]
    y = p[1]
    return [x*sin(r2) - y*cos(r2), x*cos(r2) + y*sin(r2)]

def horseshoe_transform(p):
    r = sqrt(p @ p)
    x = p[0]
    y = p[1]
    return (1/r)*array([(x-y)*(x+y), 2*x*y])

def pow_transform(p):
    x = p[0]
    y = p[1]
    r = sqrt(p @ p)
    th = arctan2(x,y)
    
    return r**sin(th)*array([cos(th), sin(th)])

def out_map(p):
    return (p+[1,1])/2

def id_map(p):
    return p

def lin_map(p, ext):
    assert(alltrue(ext>0))
    return (p+ext)/(2*ext)

In [123]:
t = 0

In [143]:
img = np.zeros((4, SIZE+1, SIZE+1), dtype=float32)
img[3,:,:].fill(1)

colors = eye(3); 

curr_col = [1,0,0] 

A = np.empty((3,2,2))
g = np.empty((3,2))

for i in range(3):
    A[i,:,:] = random.randn(2,2)
    
c = random.rand(2)*2-1
g = random.randn(3,2)
# g[0,:] = [0,0]
# g[1,:] = [1,0]
# g[2,:] = [1/2,sqrt(3)/2]

top_r = array([3,3])

for i in range(1000000):
    s = random.randint(3)
    c = A[s,:,:] @ c + g[s,:]
    c = pow_transform(c)
    
    c = maximum(c, -top_r)
    c = minimum(c, top_r)
    
    p = lin_map(c, top_r)
    
    row = SIZE-1-int(p[1]*SIZE)
    col = int(p[0]*SIZE)
    
    curr_col = (curr_col+colors[s,:])/2
    
    img[0:3,row,col] = curr_col
    img[3,row,col] += 1

for i in range(0,3):
    img[i,:,:] *= log(img[3,:,:])/img[3,:,:]
        
toimage(img[0:3,:,:]).show()
toimage(img[0:3,:,:]).save('pow_fractal{}.jpg'.format(t))
t+=1